In [3]:
import os
import json
import polars as pl

from datetime import datetime, timedelta
from pytz import timezone
from src.openmeteo import get_hist_temp

In [2]:
lf = (pl.scan_csv(f"data/eVED/*.csv")
      .select([pl.col("Matchted Latitude[deg]").alias("lat"), 
               pl.col("Matched Longitude[deg]").alias("lon")]))

In [3]:
loc_max = lf.max().collect().to_numpy()[0]

In [4]:
loc_min = lf.min().collect().to_numpy()[0]

In [5]:
loc_mid = (loc_min+loc_max)/2
locations = [
    (loc_min[0], loc_min[1]),
    (loc_min[0], loc_mid[1]),
    (loc_min[0], loc_max[1]),
    (loc_mid[0], loc_min[1]),
    (loc_mid[0], loc_mid[1]),
    (loc_mid[0], loc_max[1]),
    (loc_max[0], loc_min[1]),
    (loc_max[0], loc_mid[1]),
    (loc_max[0], loc_max[1])
]

In [6]:
locations

[(42.220268, -83.804839),
 (42.220268, -83.739138),
 (42.220268, -83.673437),
 (42.2730605, -83.804839),
 (42.2730605, -83.739138),
 (42.2730605, -83.673437),
 (42.325853, -83.804839),
 (42.325853, -83.739138),
 (42.325853, -83.673437)]

In [7]:
date_min = "2017-11-01"
date_max = "2018-12-01"

In [8]:
for i, location in enumerate(locations):
    print(i)
    filename = f"./data/openmeteo/location_{i}.json"
    if not os.path.exists(filename):
        temperatures = get_hist_temp(*location, start_date=date_min, end_date=date_max)
        with open(filename, "w") as f:
            f.write(json.dumps(temperatures))

0
1
2
3
4
5
6
7
8


In [12]:
base_dt = datetime(year=2017, month=11, day=1, tzinfo=timezone("America/Detroit"))

In [13]:
base_dt

datetime.datetime(2017, 11, 1, 0, 0, tzinfo=<DstTzInfo 'America/Detroit' LMT-1 day, 18:28:00 STD>)

In [14]:
base_dt + timedelta(days=232.672898648) + timedelta(milliseconds=0)

datetime.datetime(2018, 6, 21, 16, 8, 58, 443187, tzinfo=<DstTzInfo 'America/Detroit' LMT-1 day, 18:28:00 STD>)

In [11]:
from timezonefinder import TimezoneFinder
tf = TimezoneFinder()
tf.timezone_at(lat=42.2659958333, lng=-83.7391669444)

'America/Detroit'